[Playlist link](https://www.youtube.com/playlist?list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x)

In [1]:
import os

api_key = None
with open("../../OpenAI_API_Key.txt") as f:
    api_key = f.read().strip()
os.environ["OPENAI_API_KEY"] = api_key

# 1. Indexing

In [2]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [3]:
import tiktoken


def num_tokens_from_string(string, encoding_name):
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))


num_tokens_from_string(question, "cl100k_base")

8

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings

# embd = OllamaEmbeddings(model="phi")
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result), len(document_result)

(1536, 1536)

In [5]:
import numpy as np


# Measure Similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


print(f"similarities: {cosine_similarity(query_result, document_result)}")

similarities: 0.8807044730847652


In [5]:
# Load Document
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [8]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# 2. Retrieval

In [10]:
# Index
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=embd)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [22]:
question = "What is Task Decomposition?"
docs = retriever.get_relevant_documents(question)

# 3. Generation

In [24]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [25]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
chain = prompt | llm
chain

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f508c7539d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f508f486e10>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [26]:
chain.invoke({"context": docs, "question": docs})

AIMessage(content='Task decomposition can be done by LLM with simple prompting, task-specific instructions, or human inputs.')

## Making better Chain

In [28]:
from langchain import hub

prompt_hub_rag = hub.pull("rlm/rag-prompt")
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is task decomposition")

"Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps. It involves transforming big tasks into multiple manageable tasks to aid in the interpretation of the model's thinking process. Task decomposition can be achieved through various methods such as using prompting techniques, task-specific instructions, or human inputs."

# Example: Harry Potter Text

In [2]:
# prepare text data
text = None
with open("data/HP_1_1.txt") as f:
    text = f.read()

In [3]:
# Split using RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250, chunk_overlap=20, length_function=len
)
splits = splits = text_splitter.split_text(text)
print(len(splits))

149


In [4]:
print([len(s) for s in splits])

[247, 31, 244, 223, 248, 244, 246, 54, 248, 148, 63, 242, 108, 249, 244, 249, 136, 241, 244, 249, 246, 70, 244, 242, 234, 244, 164, 224, 241, 244, 249, 62, 194, 249, 245, 55, 67, 248, 74, 245, 34, 32, 228, 245, 94, 249, 244, 69, 241, 203, 193, 224, 151, 174, 100, 248, 22, 209, 248, 58, 185, 246, 243, 62, 22, 245, 165, 191, 243, 241, 26, 243, 233, 244, 248, 248, 109, 46, 245, 96, 189, 148, 240, 245, 25, 106, 221, 248, 78, 188, 158, 248, 228, 203, 242, 219, 247, 226, 200, 154, 99, 247, 96, 25, 244, 29, 58, 247, 237, 203, 248, 152, 161, 246, 155, 247, 88, 240, 30, 157, 166, 247, 60, 246, 202, 95, 190, 156, 242, 159, 204, 72, 224, 63, 196, 243, 248, 147, 123, 132, 177, 133, 245, 168, 98, 249, 246, 243, 67]


In [52]:
# retriever
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embd = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(splits, embedding=embd)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [53]:
retriever.get_relevant_documents("what is borrowed")

[Document(page_content='“Borrowed it, Professor Dumbledore, sir,” said the giant, climbing carefully oﬀ the motorcycle as he spoke. “Young Sirius Black lent it to me. I’ve got him, sir.”\n\n“No problems, were there?”'),
 Document(page_content='“Borrowed it, Professor Dumbledore, sir,” said the giant, climbing carefully oﬀ the motorcycle as he spoke. “Young Sirius Black lent it to me. I’ve got him, sir.”\n\n“No problems, were there?”')]

In [54]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
from langchain import hub

prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [55]:
# rag_chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

In [57]:
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [50]:
rag_chain.invoke("who is the prisoner of Azkaban?")

"I don't know."